# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
city_data_df = pd.read_csv('output_data/cities.csv')
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.19,168.86,51.01,90,1,1.01,NZ,1585575907
1,1,quzhou,28.96,118.87,50.59,85,93,2.91,CN,1585575907
2,2,paita,-5.09,-81.11,62.60,100,40,13.87,PE,1585575907
3,3,marawi,8.00,124.29,69.31,75,34,2.39,PH,1585575907
4,4,marystown,47.17,-55.15,31.68,96,99,16.02,CA,1585575907


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
#configure gmaps
gmaps.configure(api_key=g_key)


In [30]:
#Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
# Narrow down cities that fit criteria and drop any results with null values

narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"]< 80) & (city_data_df['Max Temp'] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df['Cloudiness'] == 0)].dropna()
                                       
narrowed_city_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,port alfred,-33.59,26.89,73.00,87,0,3.00,ZA,1585575908
13,13,chuy,-33.70,-53.46,78.91,64,0,6.96,UY,1585575909
53,53,sorbhog,26.50,90.87,77.59,20,0,3.60,IN,1585575919
55,55,bredasdorp,-34.53,20.04,73.00,43,0,9.17,ZA,1585575919
137,137,swellendam,-34.02,20.44,71.60,43,0,9.17,ZA,1585575940
361,361,visavadar,21.38,70.68,78.49,43,0,8.28,IN,1585575997
371,371,tura,25.52,90.22,72.70,20,0,2.80,IN,1585575999
396,396,tomatlan,19.93,-105.25,79.03,46,0,2.35,MX,1585576005
512,512,hojai,26.00,92.87,77.58,23,0,2.93,IN,1585576036


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
#Create a dataFrame called hotel_df to store names along with city, country and coordinates.

hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
8,port alfred,ZA,-33.59,26.89,
13,chuy,UY,-33.70,-53.46,
53,sorbhog,IN,26.50,90.87,
55,bredasdorp,ZA,-34.53,20.04,
137,swellendam,ZA,-34.02,20.44,
361,visavadar,IN,21.38,70.68,
371,tura,IN,25.52,90.22,
396,tomatlan,MX,19.93,-105.25,
512,hojai,IN,26.00,92.87,


In [33]:
#Set parameters to search for a hotel
params = {
    'radius': 5000,
    'types' : 'lodging',
    'key' : g_key
}

#Iterate through
for index, row in hotel_df.iterrows():
    #get lat, lng from df
    lat = row['Lat']
    lng = row['Lng']
    
    params ['location'] = f"{lat}, {lng}"
    
    #Use the search term "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make request and print url
    name_address = requests.get(base_url, params=params)
    
    #convert to JSON
    name_address = name_address.json()
    
    #Grab the first hotel from the results and store teh name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing Field/result... Skipping")

hotel_df

,City,Country,Lat,Lng,Hotel Name
8,port alfred,ZA,-33.59,26.89,The Halyards Hotel
13,chuy,UY,-33.70,-53.46,Nuevo Hotel Plaza
53,sorbhog,IN,26.50,90.87,Tourist Lodge
55,bredasdorp,ZA,-34.53,20.04,Bredasdorp Country Manor
137,swellendam,ZA,-34.02,20.44,De Kloof Luxury Estate boutique Hotel & SPA
361,visavadar,IN,21.38,70.68,Brahmanad Dham Guest House
371,tura,IN,25.52,90.22,Hotel RIKMAN Continental
396,tomatlan,MX,19.93,-105.25,Norma franco
512,hojai,IN,26.00,92.87,Luxury park hotel


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))